# Lesson 5: Human in the Loop

Note: This notebook is running in a later version of langgraph that it was filmed with. The later version has a couple of key additions:
- Additional state information is stored to memory and displayed when using `get_state()` or `get_state_history()`.
- State is additionally stored every state transition while previously it was stored at an interrupt or at the end.
These change the command output slightly, but are a useful addtion to the information available.

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_ollama import ChatOllama
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
from uuid import uuid4

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [65]:
tool = DuckDuckGoSearchRun()

## Manual human approval

In [66]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_model)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_model(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def _sanitize_args(self, args):
        """Local LLMs sometimes wrap arg values as {'description': ..., 'value': ...}"""
        sanitized = {}
        for k, v in args.items():
            if isinstance(v, dict):
                sanitized[k] = v.get('value', v.get('description', str(v)))
            else:
                sanitized[k] = v
        return sanitized

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            args = self._sanitize_args(t['args'])
            result = self.tools[t['name']].invoke(args)
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [67]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOllama(model="lfm2.5-thinking", temperature=0)
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [68]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='b1c5d640-6f13-4e13-8b53-e81f35de0a24'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:48.890307Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2104079208, 'load_duration': 61533083, 'prompt_eval_count': 176, 'prompt_eval_duration': 164055708, 'eval_count': 363, 'eval_duration': 1810777303, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-4efa-7e42-855f-52d0196927cf-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'weather in San Francisco'}, 'id': '1e51847e-00e6-4c4f-9d47-439ee0a5acb2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 363, 'total_tokens': 539})]}
{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created

In [69]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='b1c5d640-6f13-4e13-8b53-e81f35de0a24'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:48.890307Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2104079208, 'load_duration': 61533083, 'prompt_eval_count': 176, 'prompt_eval_duration': 164055708, 'eval_count': 363, 'eval_duration': 1810777303, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-4efa-7e42-855f-52d0196927cf-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'weather in San Francisco'}, 'id': '1e51847e-00e6-4c4f-9d47-439ee0a5acb2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 363, 'total_tokens': 539})]}, next=('action',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoi

In [70]:
abot.graph.get_state(thread).next

('action',)

### continue after interrupt

In [71]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'duckduckgo_search', 'args': {'query': 'weather in San Francisco'}, 'id': '1e51847e-00e6-4c4f-9d47-439ee0a5acb2', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content="Explanation of the causes of San Francisco's summer climate feeling cool and foggy, including frequent Karl-the-Fog roll-ins that create a damp, windy feel, often dropping perceived temperatures into the 50s–60s even on warm days. 5:09 PM. San Francisco Weather Radar. San Francisco Weather Radar. Static Radar Temporarily Unavailable. Thank you for your patience as we work to get everything up and running again. San Francisco - Weather warnings issued. 14-day forecast. Add to your locationsAdd to your locations.Pressure: Pressure: 1019 millibars, Falling,1019mb, Falling. Observation station: San Francisco /Int. Current local time in USA – California – San Francisco . Get San Francisco 's weather and area codes, time zone and DST. Weather for more locations. Vacation. Schools.Canada's

In [72]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='b1c5d640-6f13-4e13-8b53-e81f35de0a24'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:48.890307Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2104079208, 'load_duration': 61533083, 'prompt_eval_count': 176, 'prompt_eval_duration': 164055708, 'eval_count': 363, 'eval_duration': 1810777303, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-4efa-7e42-855f-52d0196927cf-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'weather in San Francisco'}, 'id': '1e51847e-00e6-4c4f-9d47-439ee0a5acb2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 363, 'total_tokens': 539}), ToolMessage(content="Explanation of the causes of San Francisco's summer climate feeling cool

In [73]:
abot.graph.get_state(thread).next

('action',)

In [74]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='3e71e309-1050-4858-9272-29c2bf6df22e'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:53.226277Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2093328917, 'load_duration': 71136459, 'prompt_eval_count': 176, 'prompt_eval_duration': 17457542, 'eval_count': 424, 'eval_duration': 1940200457, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-5ffb-7153-a561-a14949547055-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather in Los Angeles'}, 'id': '4fb9d644-b4be-40d7-b63a-59d99bfddcdd', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 424, 'total_tokens': 600})]}
{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'cr

## Modify State
Run until the interrupt and then modify the state.

In [75]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='0486a9fb-063f-4b0a-ab1a-e03f5a5ec34c'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:57.615286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 684148833, 'load_duration': 67154333, 'prompt_eval_count': 176, 'prompt_eval_duration': 37251709, 'eval_count': 128, 'eval_duration': 561347876, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-76a2-7cb1-bb1c-1bb1713f9205-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather in Los Angeles'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 128, 'total_tokens': 304})]}
{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'crea

In [76]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='0486a9fb-063f-4b0a-ab1a-e03f5a5ec34c'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:57.615286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 684148833, 'load_duration': 67154333, 'prompt_eval_count': 176, 'prompt_eval_duration': 37251709, 'eval_count': 128, 'eval_duration': 561347876, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-76a2-7cb1-bb1c-1bb1713f9205-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather in Los Angeles'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 128, 'total_tokens': 304})]}, next=('action',), config={'configurable': {'thread_id': '3', 'checkpoint_ns': '', 'check

In [77]:
current_values = abot.graph.get_state(thread)

In [78]:
current_values.values['messages'][-1]

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:57.615286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 684148833, 'load_duration': 67154333, 'prompt_eval_count': 176, 'prompt_eval_duration': 37251709, 'eval_count': 128, 'eval_duration': 561347876, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-76a2-7cb1-bb1c-1bb1713f9205-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather in Los Angeles'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 128, 'total_tokens': 304})

In [79]:
current_values.values['messages'][-1].tool_calls

[{'name': 'duckduckgo_search',
  'args': {'query': 'current weather in Los Angeles'},
  'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2',
  'type': 'tool_call'}]

In [80]:
_id = current_values.values['messages'][-1].tool_calls[0]['id']
current_values.values['messages'][-1].tool_calls = [
    {'name': 'duckduckgo_search',
  'args': {'query': 'current weather in Louisiana'},
  'id': _id}
]

In [81]:
abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='0486a9fb-063f-4b0a-ab1a-e03f5a5ec34c'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:57.615286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 684148833, 'load_duration': 67154333, 'prompt_eval_count': 176, 'prompt_eval_duration': 37251709, 'eval_count': 128, 'eval_duration': 561347876, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-76a2-7cb1-bb1c-1bb1713f9205-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather in Louisiana'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 128, 'total_tokens': 304})]}


{'configurable': {'thread_id': '3',
  'checkpoint_ns': '',
  'checkpoint_id': '1f10c1ce-ba06-6b6a-8002-cf01951e9077'}}

In [82]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='0486a9fb-063f-4b0a-ab1a-e03f5a5ec34c'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:57.615286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 684148833, 'load_duration': 67154333, 'prompt_eval_count': 176, 'prompt_eval_duration': 37251709, 'eval_count': 128, 'eval_duration': 561347876, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-76a2-7cb1-bb1c-1bb1713f9205-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather in Louisiana'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 128, 'total_tokens': 304})]}, next=('action',), config={'configurable': {'thread_id': '3', 'checkpoint_ns': '', 'checkpo

In [83]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'duckduckgo_search', 'args': {'query': 'current weather in Louisiana'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='The information here tells how often heat combines with humidity in Louisiana cities to create uncomfortably muggy weather . ... in , primarily overcast and occasionally bright ... California Weather Alert: Flooding in Redding, Heavy Rain and Sierra Snow During Holiday Travel Bear in mind that direct sunshine exposure increases weather impact and may raise the heat index by up to 15 Fahrenheit (8 Celsius) degrees. Current Louisiana Weather Conditions, Weather Radar, and Forecast as Reported in the Center of the State at Alexandria (courtesy of Accuweather) There, you will find tailored advice to help you stay comfortable and prepared for the weather conditions in Louisiana today.', name='duckduckgo_search', id='a0c1e799-4d55-49cb-99d5-64cf15a9a5d0', tool_call_id='3190d4e5-1b30-4

## Time Travel

In [84]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print('--')
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='0486a9fb-063f-4b0a-ab1a-e03f5a5ec34c'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:57.615286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 684148833, 'load_duration': 67154333, 'prompt_eval_count': 176, 'prompt_eval_duration': 37251709, 'eval_count': 128, 'eval_duration': 561347876, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-76a2-7cb1-bb1c-1bb1713f9205-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather in Louisiana'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 128, 'total_tokens': 304}), ToolMessage(content='The information here tells how often heat combines with humidity in Lou

To fetch the same state as was filmed, the offset below is changed to `-3` from `-1`. This accounts for the initial state `__start__` and the first state that are now stored to state memory with the latest version of software.

In [85]:
to_replay = states[-3]

In [86]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='0486a9fb-063f-4b0a-ab1a-e03f5a5ec34c'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:57.615286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 684148833, 'load_duration': 67154333, 'prompt_eval_count': 176, 'prompt_eval_duration': 37251709, 'eval_count': 128, 'eval_duration': 561347876, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-76a2-7cb1-bb1c-1bb1713f9205-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather in Los Angeles'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 128, 'total_tokens': 304})]}, next=('action',), config={'configurable': {'thread_id': '3', 'checkpoint_ns': '', 'check

In [87]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

Calling: {'name': 'duckduckgo_search', 'args': {'query': 'current weather in Los Angeles'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content="Use Current Location. Recent. Los Angeles . California. 14°. Los Angeles Weather Radar. Static Radar Temporarily Unavailable. Thank you for your patience as we work to get everything up and running again. Current local time in USA – California – Los Angeles . Get Los Angeles 's weather and area codes, time zone and DST. Photo from Los Angeles representing current weather conditions.Even warmer weather awaits central, eastern US later in February. Los Angeles - Weather warnings issued. 14-day forecast.Forecast - Los Angeles . Day by day forecast. Last updated today at 09:02. Los Angeles , CA The current weather condition in Los Angeles is Sunny and 67°. 67°. Los Angeles weather report and traffic updates by locals. Write your own Los Angeles , California report, forecast, or tr

## Go back in time and edit

In [88]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='0486a9fb-063f-4b0a-ab1a-e03f5a5ec34c'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:57.615286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 684148833, 'load_duration': 67154333, 'prompt_eval_count': 176, 'prompt_eval_duration': 37251709, 'eval_count': 128, 'eval_duration': 561347876, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-76a2-7cb1-bb1c-1bb1713f9205-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather in Los Angeles'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 128, 'total_tokens': 304})]}, next=('action',), config={'configurable': {'thread_id': '3', 'checkpoint_ns': '', 'check

In [89]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']
to_replay.values['messages'][-1].tool_calls = [{'name': 'duckduckgo_search',
  'args': {'query': 'current weather in LA, accuweather'},
  'id': _id}]

In [90]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='0486a9fb-063f-4b0a-ab1a-e03f5a5ec34c'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:57.615286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 684148833, 'load_duration': 67154333, 'prompt_eval_count': 176, 'prompt_eval_duration': 37251709, 'eval_count': 128, 'eval_duration': 561347876, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-76a2-7cb1-bb1c-1bb1713f9205-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather in LA, accuweather'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 128, 'total_tokens': 304})]}


In [91]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling: {'name': 'duckduckgo_search', 'args': {'query': 'current weather in LA, accuweather'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='This interactive map provides live current temperatures in and around your area. ... AccuWeather " and sun design are registered trademarks of ... Older adults, infants, and those with sensitive medical ... AccuWeather " and sun design are registered trademarks of AccuWeather , Inc. AccuWeather Ready Business Health Hurricane Leisure and ... AccuWeather " and sun design are registered trademarks of AccuWeather , Inc. AccuWeather Ready Business Health Hurricane Leisure and ... AccuWeather " and sun design are registered trademarks of AccuWeather , Inc. ... AccuWeather APIs AccuWeather Connect Personal Weather Stations ... AccuWeather " and sun design are registered trademarks of AccuWeather , Inc.', name='duckduckgo_search', id='bd462778-b0ea-4536-b06d-9950be46adb6', tool_c

## Add message to a state at a given time

In [92]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='0486a9fb-063f-4b0a-ab1a-e03f5a5ec34c'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:57.615286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 684148833, 'load_duration': 67154333, 'prompt_eval_count': 176, 'prompt_eval_duration': 37251709, 'eval_count': 128, 'eval_duration': 561347876, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-76a2-7cb1-bb1c-1bb1713f9205-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather in LA, accuweather'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 128, 'total_tokens': 304})]}, next=('action',), config={'configurable': {'thread_id': '3', 'checkpoint_ns': '', 'checkpoint_id': '1f10c

In [93]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']

In [94]:
state_update = {"messages": [ToolMessage(
    tool_call_id=_id,
    name="duckduckgo_search",
    content="54 degree celcius",
)]}

In [95]:
branch_and_add = abot.graph.update_state(
    to_replay.config, 
    state_update, 
    as_node="action")

In [96]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='0486a9fb-063f-4b0a-ab1a-e03f5a5ec34c'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-17T16:22:57.615286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 684148833, 'load_duration': 67154333, 'prompt_eval_count': 176, 'prompt_eval_duration': 37251709, 'eval_count': 128, 'eval_duration': 561347876, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c6c69-76a2-7cb1-bb1c-1bb1713f9205-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather in Los Angeles'}, 'id': '3190d4e5-1b30-46ba-be41-e6672b82d6d2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 128, 'total_tokens': 304}), ToolMessage(content='54 degree celcius', name='duckduckgo_search', id='ba5a5346-a681-48fd-9eec-ef94da8c57e9', t

# Extra Practice

## Build a small graph
This is a small simple graph you can tinker with if you want more insight into controlling state memory.

In [97]:
from typing import TypedDict, Annotated
import operator
from langgraph.checkpoint.memory import MemorySaver

Define a simple 2 node graph with the following state:
-`lnode`: last node
-`scratch`: a scratchpad location
-`count` : a counter that is incremented each step

In [98]:
class AgentState(TypedDict):
    lnode: str
    scratch: str
    count: Annotated[int, operator.add]

In [99]:
def node1(state: AgentState):
    print(f"node1, count:{state['count']}")
    return {"lnode": "node_1",
            "count": 1,
           }
def node2(state: AgentState):
    print(f"node2, count:{state['count']}")
    return {"lnode": "node_2",
            "count": 1,
           }

The graph goes N1->N2->N1... but breaks after count reaches 3.

In [100]:
def should_continue(state):
    return state["count"] < 3

In [101]:
builder = StateGraph(AgentState)
builder.add_node("Node1", node1)
builder.add_node("Node2", node2)

builder.add_edge("Node1", "Node2")
builder.add_conditional_edges("Node2", 
                              should_continue, 
                              {True: "Node1", False: END})
builder.set_entry_point("Node1")

In [102]:
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

### Run it!
Now, set the thread and run!

In [103]:
thread = {"configurable": {"thread_id": str(1)}}
graph.invoke({"count":0, "scratch":"hi"},thread)

node1, count:0
node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

### Look at current state

Get the current state. Note the `values` which are the AgentState. Note the `config` and the `thread_ts`. You will be using those to refer to snapshots below.

In [104]:
graph.get_state(thread)

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c4b-652e-8004-13ee28afb32b'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2026-02-17T16:23:14.712292+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c49-60ee-8003-8ed336c81045'}}, tasks=(), interrupts=())

View all the statesnapshots in memory. You can use the displayed `count` agentstate variable to help track what you see. Notice the most recent snapshots are returned by the iterator first. Also note that there is a handy `step` variable in the metadata that counts the number of steps in the graph execution. This is a bit detailed - but you can also notice that the *parent_config* is the *config* of the previous node. At initial startup, additional states are inserted into memory to create a parent. This is something to check when you branch or *time travel* below.

### Look at state history

In [105]:
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c4b-652e-8004-13ee28afb32b'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2026-02-17T16:23:14.712292+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c49-60ee-8003-8ed336c81045'}}, tasks=(), interrupts=()) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c49-60ee-8003-8ed336c81045'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2026-02-17T16:23:14.711363+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c48-6040-8002-a75922c66f1d'}}, tasks=(PregelTask(id='fa57030c-c187-9030-192d-f40ecb6039fe', name='Node2', path=('__pregel_pull

Store just the `config` into an list. Note the sequence of counts on the right. `get_state_history` returns the most recent snapshots first.

In [106]:
states = []
for state in graph.get_state_history(thread):
    states.append(state.config)
    print(state.config, state.values['count'])

{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c4b-652e-8004-13ee28afb32b'}} 4
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c49-60ee-8003-8ed336c81045'}} 3
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c48-6040-8002-a75922c66f1d'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c40-6fde-8001-cbde5e0773a8'}} 1
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c03-66e8-8000-ba8b7eb51bea'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c01-680c-bfff-2a6bfc8a2e8c'}} 0


Grab an early state.

In [107]:
states[-3]

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f10c1cf-5c40-6fde-8001-cbde5e0773a8'}}

This is the state after Node1 completed for the first time. Note `next` is `Node2`and `count` is 1.

In [108]:
graph.get_state(states[-3])

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c40-6fde-8001-cbde5e0773a8'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2026-02-17T16:23:14.708053+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c03-66e8-8000-ba8b7eb51bea'}}, tasks=(PregelTask(id='29846ed6-4dfe-9af9-c5e8-e248077e2746', name='Node2', path=('__pregel_pull', 'Node2'), error=None, interrupts=(), state=None, result={'lnode': 'node_2', 'count': 1}),), interrupts=())

### Go Back in Time
Use that state in `invoke` to go back in time. Notice it uses states[-3] as *current_state* and continues to node2,

In [109]:
graph.invoke(None, states[-3])

node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

Notice the new states are now in state history. Notice the counts on the far right.

In [110]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state.config, state.values['count'])

{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5cdb-6840-8004-3710583cbdfc'}} 4
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5cda-6eae-8003-c0f5af1a47e0'}} 3
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5cd8-6c58-8002-1c96fc8c1371'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c4b-652e-8004-13ee28afb32b'}} 4
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c49-60ee-8003-8ed336c81045'}} 3
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c48-6040-8002-a75922c66f1d'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c40-6fde-8001-cbde5e0773a8'}} 1
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5c03-66e8-8000-ba8b7eb51bea'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkp

You can see the details below. Lots of text, but try to find the node that start the new branch. Notice the parent *config* is not the previous entry in the stack, but is the entry from state[-3].

In [111]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state,"\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5cdb-6840-8004-3710583cbdfc'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2026-02-17T16:23:14.771356+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5cda-6eae-8003-c0f5af1a47e0'}}, tasks=(), interrupts=()) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5cda-6eae-8003-c0f5af1a47e0'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2026-02-17T16:23:14.771108+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5cd8-6c58-8002-1c96fc8c1371'}}, tasks=(PregelTask(id='b72c2d79-cf60-8f32-9427-28bcc9ea37ac', name='Node2', path=('__pregel_pull

### Modify State
Let's start by starting a fresh thread and running to clean out history.

In [112]:
thread2 = {"configurable": {"thread_id": str(2)}}
graph.invoke({"count":0, "scratch":"hi"},thread2)

node1, count:0
node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [113]:
states2 = []
for state in graph.get_state_history(thread2):
    states2.append(state.config)
    print(state.config, state.values['count'])   

{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d38-67de-8004-46ff6025a5b0'}} 4
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d36-6524-8003-d333917bb4cc'}} 3
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d35-6764-8002-4f309c7f1995'}} 2
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d33-68c4-8001-5cbd9e0f68a5'}} 1
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d31-6f7e-8000-6ebb4207f9e9'}} 0
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d30-6750-bfff-f09765afc263'}} 0


Start by grabbing a state.

In [114]:
save_state = graph.get_state(states2[-3])
save_state

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d33-68c4-8001-5cbd9e0f68a5'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2026-02-17T16:23:14.807365+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d31-6f7e-8000-6ebb4207f9e9'}}, tasks=(PregelTask(id='44c97efb-179a-6dfb-7a7b-41a7cbf0b9a2', name='Node2', path=('__pregel_pull', 'Node2'), error=None, interrupts=(), state=None, result={'lnode': 'node_2', 'count': 1}),), interrupts=())

Now modify the values. One subtle item to note: Recall when agent state was defined, `count` used `operator.add` to indicate that values are *added* to the current value. Here, `-3` will be added to the current count value rather than replace it.

In [115]:
save_state.values["count"] = -3
save_state.values["scratch"] = "hello"
save_state

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hello', 'count': -3}, next=('Node2',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d33-68c4-8001-5cbd9e0f68a5'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2026-02-17T16:23:14.807365+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d31-6f7e-8000-6ebb4207f9e9'}}, tasks=(PregelTask(id='44c97efb-179a-6dfb-7a7b-41a7cbf0b9a2', name='Node2', path=('__pregel_pull', 'Node2'), error=None, interrupts=(), state=None, result={'lnode': 'node_2', 'count': 1}),), interrupts=())

Now update the state. This creates a new entry at the *top*, or *latest* entry in memory. This will become the current state.

In [116]:
graph.update_state(thread2,save_state.values)

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f10c1cf-5d93-6f62-8005-51faff5f3556'}}

Current state is at the top. You can match the `thread_ts`.
Notice the `parent_config`, `thread_ts` of the new node - it is the previous node.

In [117]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:  #print latest 3
        break
    print(state, '\n')

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hello', 'count': 1}, next=('Node1',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d93-6f62-8005-51faff5f3556'}}, metadata={'source': 'update', 'step': 5, 'parents': {}}, created_at='2026-02-17T16:23:14.846902+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d38-67de-8004-46ff6025a5b0'}}, tasks=(PregelTask(id='e69584f1-fc0b-51a1-307a-e31a4e0c757d', name='Node1', path=('__pregel_pull', 'Node1'), error=None, interrupts=(), state=None, result=None),), interrupts=()) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d38-67de-8004-46ff6025a5b0'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2026-02-17T16:23:14.809437+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': 

### Try again with `as_node`
When writing using `update_state()`, you want to define to the graph logic which node should be assumed as the writer. What this does is allow th graph logic to find the node on the graph. After writing the values, the `next()` value is computed by travesing the graph using the new state. In this case, the state we have was written by `Node1`. The graph can then compute the next state as being `Node2`. Note that in some graphs, this may involve going through conditional edges!  Let's try this out.

In [118]:
graph.update_state(thread2,save_state.values, as_node="Node1")

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f10c1cf-5dc2-67b8-8006-82480d9152e0'}}

In [119]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:  #print latest 3
        break
    print(state, '\n')

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hello', 'count': -2}, next=('Node2',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5dc2-67b8-8006-82480d9152e0'}}, metadata={'source': 'update', 'step': 6, 'parents': {}}, created_at='2026-02-17T16:23:14.865958+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d93-6f62-8005-51faff5f3556'}}, tasks=(PregelTask(id='91d9e8fa-2145-96f3-449c-7ada5f7b2d1d', name='Node2', path=('__pregel_pull', 'Node2'), error=None, interrupts=(), state=None, result=None),), interrupts=()) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hello', 'count': 1}, next=('Node1',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5d93-6f62-8005-51faff5f3556'}}, metadata={'source': 'update', 'step': 5, 'parents': {}}, created_at='2026-02-17T16:23:14.846902+00:00', parent_config={'configurable': {'thread_id': '2', 'ch

`invoke` will run from the current state if not given a particular `thread_ts`. This is now the entry that was just added.

In [120]:
graph.invoke(None,thread2)

node2, count:-2
node1, count:-1
node2, count:0
node1, count:1
node2, count:2


{'lnode': 'node_2', 'scratch': 'hello', 'count': 3}

Print out the state history, notice the `scratch` value change on the latest entries.

In [121]:
for state in graph.get_state_history(thread2):
    print(state,"\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hello', 'count': 3}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5e0b-6f4e-800b-4b37e94cf97f'}}, metadata={'source': 'loop', 'step': 11, 'parents': {}}, created_at='2026-02-17T16:23:14.896043+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5e0b-615c-800a-c3c457d0174a'}}, tasks=(), interrupts=()) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hello', 'count': 2}, next=('Node2',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5e0b-615c-800a-c3c457d0174a'}}, metadata={'source': 'loop', 'step': 10, 'parents': {}}, created_at='2026-02-17T16:23:14.895677+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f10c1cf-5e08-68a8-8009-14081de81c81'}}, tasks=(PregelTask(id='74ff0ccb-6fd2-9170-656c-231b88f76e9f', name='Node2', path=('__pre

Continue to experiment!